In [33]:
# ref https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/rankGPT/

In [3]:
# %pip install llama-index-postprocessor-rankgpt-rerank
# %pip install llama-index-llms-huggingface
# %pip install llama-index-llms-huggingface-api
# %pip install llama-index-llms-openai
# %pip install llama-index-llms-ollama

In [1]:
from dotenv import load_dotenv
import os
import sys
import logging

from llama_index.core import StorageContext, load_index_from_storage

from llama_index.core.retrievers import VectorIndexRetriever

from llama_index.core import Settings

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
import openai

import nest_asyncio
nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

load_dotenv()  # Load environment variables from .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = openai.api_key

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


%load_ext autoreload
%autoreload 2




In [2]:


# Rebuild the storage context
storage_context = StorageContext.from_defaults(persist_dir="./persist")

# Load the index from storage
index = load_index_from_storage(storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [20]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

import pandas as pd
from IPython.display import display, HTML


def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = RankGPTRerank(
            llm=OpenAI(
               model="gpt-3.5-turbo",
               temperature=0.0,
               api_key=OPENAI_API_KEY,
            ),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [21]:
retrieved_nodes = get_retrieved_nodes(
    "What are potential CONSECUENCES of keytruda?",
    vector_top_k=25,
    with_reranker=False,
)

# for idx, node in enumerate(retrieved_nodes):
#     print(f"Retrieved Node {idx}: Embedding:  {node.embedding}, embedding_VDB: {index.vector_store.get(node.id_)}")


visualize_retrieved_nodes(retrieved_nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


,Score,Text
0,0.694642,"Can Keytruda cause immune-related adverse effects? Yes, Keytruda can cause immune-related adverse effects such as colitis, hepatitis, and pneumonitis."
1,0.655332,"Are there specific side effects of Keytruda that NSCLC patients should monitor? NSCLC patients should monitor for cough, shortness of breath, and chest pain, as these could indicate immune-related pneumonitis."
2,0.643183,"What should patients report immediately while on Keytruda treatment? Patients should report any new or worsening symptoms such as cough, chest pain, or changes in vision immediately."
3,0.632885,Can Keytruda be used in NSCLC patients with autoimmune diseases? Keytruda should be used with caution in patients with pre-existing autoimmune diseases due to the risk of exacerbating the condition.
4,0.631243,"Can Keytruda cause changes in blood pressure? Yes, fluctuations in blood pressure can occur, and patients should monitor their blood pressure regularly."
5,0.630975,"What are the common side effects of Keytruda? Common side effects include fatigue, nausea, and skin rash."
6,0.630753,How does Keytruda affect immune system function in NSCLC patients? Keytruda enhances the immune system's ability to detect and destroy cancer cells but can also lead to immune-related adverse effects that need to be managed.
7,0.612230,"Are there any known interactions between Keytruda and other medications? Yes, Keytruda can interact with steroids and certain immunosuppressants, potentially affecting its efficacy and safety."
8,0.602651,"Are there any specific monitoring protocols for NSCLC patients on Keytruda as suggested by KEYNOTE-006 outcomes? Regular monitoring of liver function, lung function, and immune markers is recommended to manage potential side effects effectively."
9,0.600999,"Can NSCLC patients with brain metastases be treated with Keytruda? Keytruda can be used in NSCLC patients with brain metastases, especially when systemic disease control is needed, but close monitoring for neurological symptoms is essential."


In [22]:
retrieved_nodes = get_retrieved_nodes(
    "What are potential CONSECUENCES of keytruda?",
    vector_top_k=25,
    reranker_top_n=3,
    with_reranker=True,
)

# for idx, node in enumerate(retrieved_nodes):
#     print(f"Retrieved Node {idx}: Embedding:  {node.embedding}, embedding_VDB: {index.vector_store.get(node.id_)}")

visualize_retrieved_nodes(retrieved_nodes)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
After Reranking, new rank list for nodes: [0, 6, 3, 7, 1, 8, 9, 13, 14, 16, 20, 18, 23, 24, 4, 10, 5, 2, 11, 21, 19, 17, 12, 15, 22]

,Score,Text
0,0.694664,"Can Keytruda cause immune-related adverse effects? Yes, Keytruda can cause immune-related adverse effects such as colitis, hepatitis, and pneumonitis."
1,0.630779,How does Keytruda affect immune system function in NSCLC patients? Keytruda enhances the immune system's ability to detect and destroy cancer cells but can also lead to immune-related adverse effects that need to be managed.
2,0.632893,Can Keytruda be used in NSCLC patients with autoimmune diseases? Keytruda should be used with caution in patients with pre-existing autoimmune diseases due to the risk of exacerbating the condition.
